In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None) ## Couldn't see the list I made inside columns.

import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
test_file = '/kaggle/input/playground-series-s3e22/test.csv'
train_file = '/kaggle/input/playground-series-s3e22/train.csv'

In [ ]:
test_data = pd.read_csv(test_file)
train_data = pd.read_csv(train_file)

# drop_outcome_df = train_data.copy()
# drop_outcome_df = drop_outcome_df.drop(['outcome'],axis=1)

In [ ]:
test_data

In [ ]:
train_data

In [ ]:
train_data.dtypes

In [ ]:
##prints out the unique columns

# string_columns = train_data.select_dtypes(include=['object'])

# unique_words = {}
# for column in string_columns.columns:
#     unique_values = string_columns[column].unique()
#     unique_words[column] = unique_values
    
# for column, unique_values in unique_words.items():
#     print(f'Unique values in {column}:')
#     for value in unique_values:
#         print(value)
#     print()

In [ ]:
table = pd.DataFrame({"Feature":[],"missing":[],"unique":[]})

for column in train_data.columns:
    table.loc[len(table.index)] = [column,sum(pd.isnull(x) for x in train_data[column]),len(set(x for x in train_data[column]))]
    
display(table)
# null_train_values = train_data.isnull().sum()
# null_train_values

In [ ]:
string_columns = train_data.select_dtypes(include=['object'])

unique_values_dict = {}

for column in string_columns.columns:
    uv = string_columns[column].unique()
    unique_values_dict[column] = [uv.tolist()]
    
unique_values_df = pd.DataFrame.from_dict(unique_values_dict)

#############################################################

string_columns = test_data.select_dtypes(include=['object'])

unique_values_dict = {}

for column in string_columns.columns:
    uv = string_columns[column].unique()
    unique_values_dict[column] = [uv.tolist()]
    
test_unique_values_df = pd.DataFrame.from_dict(unique_values_dict)    

In [ ]:
unique_values_df

In [ ]:
test_unique_values_df

In [ ]:
test_data_column_list = test_data.columns.tolist()
train_data_column_list = train_data.columns.tolist()

In [ ]:
diff = pd.DataFrame()

for column in test_data_column_list:
    differences = train_data[column][0] != test_data[column][0]
    diff[column] = differences
    
diff

In [ ]:
len(test_data_column_list)

In [ ]:
len(train_data_column_list)

In [ ]:
train_data_column_list.remove('outcome')

In [ ]:
different_columns = set(train_data.columns) - set(test_data.columns)

different_columns

In [ ]:
if train_data_column_list == test_data_column_list:
    print('True')

### There are several things that can be chaged in these dataframes.
#### These note are more for my benefit but will be left here

##### capillary 
time can be numeric  0, 2,3,4
##### pain 
column has moderate in test only
##### peristalsis
train has distend_small
##### rectal_exam_feces
has serosanguious in train

Just found out that some horses died twice.  Do not drop those horses though as it significantly impairs the calculation.

In [ ]:
## Replacing object nan_values with no_measure_take.  Its not clear if a measure was taken or not but in some instances it appears a measure was not needed

def replace_nans_values(dataframe):
    object_columns = dataframe.select_dtypes(include='object').columns
    numeric_columns = dataframe.select_dtypes(include=['int64', 'float64']).columns
    
    dataframe[object_columns] = dataframe[object_columns].fillna('no_measure_taken')
    dataframe[numeric_columns] = dataframe[numeric_columns].fillna(0)
    
    return dataframe

train_data = replace_nans_values(train_data)
test_data = replace_nans_values(test_data)

# train_data['hospital_number'] = train_data['hospital_number'].astype('object')
# test_data['hospital_number'] = test_data['hospital_number'].astype('object')



## Make int64 and float64 nan to 0   



## Make Capillary object.  change nan to no measure take

## Keep Capillary but change nan to no measure taken

## remove distend_small from training
count_distend_small = train_data[train_data['peristalsis'].str.contains('distend_small')].shape[0]
print(f'Rows with distend_small:  {count_distend_small}')
train_data['peristalsis'] = train_data['peristalsis'].str.replace('distend_small','normal')

## remove serosanguious in training
count_serosanguious = train_data[train_data['rectal_exam_feces'].str.contains('serosanguious')].shape[0]
print(f'Rows with serosanguious: {count_serosanguious}')
train_data['rectal_exam_feces'] = train_data['rectal_exam_feces'].str.replace('serosanguious','normal')

## remove moderate from test
test_data['pain'] = test_data['pain'].str.replace('moderate','mild_pain')



In [ ]:
numerical_columns = train_data.select_dtypes(include=['int64', 'float64']).drop(columns=['id'])

numerical_columns.hist(figsize=(16,20),bins=40, xlabelsize=10, ylabelsize=10)



In [ ]:
grouping = train_data.groupby(['age','surgery','outcome']).size().unstack(fill_value=0)

ax = grouping.plot(kind='bar', stacked=True, figsize=(10,6))

ax.set_xlabel('Age and Surgery')
ax.set_ylabel('Count')
ax.set_title('Outcome by Age and Surgery')
plt.xticks(rotation=0)
plt.legend(title='Outcome')
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12,8), gridspec_kw={'hspace':0.4})

fig.suptitle("Distribution of Numerical Features by Outcomes; Probability Density")

sns.violinplot(x='outcome',y='rectal_temp',data=train_data, inner='quart', palette='Set1',ax=axes[0,0])
axes[0,0].set_title('Distribution of Rectal Temps by Outcome',fontsize=9)
axes[0,0].set_xlabel('Outcome')
axes[0,0].set_ylabel('Rectal Temp')

sns.violinplot(x='outcome',y='pulse',data=train_data, inner='quart', palette='Set1',ax=axes[0,1])
axes[0,1].set_title('Distribution of Pulse Temps by Outcome',fontsize=9)
axes[0,1].set_xlabel('Outcome')
axes[0,1].set_ylabel('Pulse')

sns.violinplot(x='outcome',y='total_protein',data=train_data, inner='quart', palette='Set1',ax=axes[0,2])
axes[0,2].set_title('Distribution of Total_Proten Temps by Outcome',fontsize=9)
axes[0,2].set_xlabel('Outcome')
axes[0,2].set_ylabel('Total Protien')

sns.violinplot(x='outcome',y='respiratory_rate',data=train_data, inner='quart', palette='Set1',ax=axes[1,0])
axes[1,0].set_title('Distribution of Respiratory Rate Temps by Outcome',fontsize=9)
axes[1,0].set_xlabel('Outcome')
axes[1,0].set_ylabel('Respiratory Rate')

sns.violinplot(x='outcome',y='packed_cell_volume',data=train_data, inner='quart', palette='Set1',ax=axes[1,1])
axes[1,1].set_title('Distribution of Packed Cell Volumn Temps by Outcome',fontsize=8)
axes[1,1].set_xlabel('Outcome')
axes[1,1].set_ylabel('Packed Cell Volume')

sns.violinplot(x='outcome',y='abdomo_protein',data=train_data, inner='quart', palette='Set1',ax=axes[1,2])
axes[1,2].set_title('Distribution of Abdomo Protein by Outcome',fontsize=9)
axes[1,2].set_xlabel('Outcome')
axes[1,2].set_ylabel('Abdomo Protein')



In [ ]:
# object_counts = train_data.apply(pd.value_counts).fillna(0).T
# ax = object_counts.plot(kind='bar',stacked=True,figsize=(10,6))

# melted_df = pd.melt(train_data)
# ax = melted_df['value'].value_counts().unstack().fillna(0).T.plot(kind='bar',stacked=True,figsize=(10,6))

ct = pd.crosstab(train_data['peripheral_pulse'],train_data['pain'])

ax = ct.plot(kind='bar', stacked=True, figsize=(10,6))

plt.title("Stacked Bar Chart of Categorical Features")
plt.xlabel("Features")
plt.ylabel("Counts")
plt.legend(title="Categories")
plt.show()

In [ ]:
## This is heavily borrowed and requires research
## Cramers V statistic - Association between categorical variables

## Works but produces a runtime error better to OneHotEncode

# from scipy.stats import chi2_contingency
# def cramers_v(x, y):
#     confusion_matrix = pd.crosstab(x, y)
#     chi2 = chi2_contingency(confusion_matrix)[0]
#     n = confusion_matrix.sum().sum()
#     phi2 = chi2/n
#     r, k = confusion_matrix.shape
#     phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
#     rcorr = r - ((r - 1) ** 2) / (n - 1)
#     kcorr = k - ((k - 1) ** 2) / (n - 1)
#     return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

# corr_matrix = np.zeros((len(train_data.columns), len(train_data.columns)))
# columns = train_data.columns

# for i in range(len(columns)):
#     for j in range(len(columns)):
#         if i != j:
#             corr_matrix[i, j] = cramers_v(train_data[columns[i]], train_data[columns[j]])

# plt.figure(figsize=(10, 6))
# sns.set(font_scale=1.2)
# sns.heatmap(corr_matrix, annot=True, fmt=".2f", xticklabels=columns, yticklabels=columns, cmap="coolwarm", linewidths=0.5)
# plt.title("Categorical Feature Correlation (Cramer's V)")
# plt.show()

# Feature Selections

In [ ]:
X_train = train_data.drop(['id','outcome','lesion_3'],axis=1)
y_train = train_data['outcome']
X_test = test_data.drop(['id','lesion_3'],axis=1)

#object_columns = [oname for oname in X_train.columns if
                 #X_train[oname].dtype == 'object']
    
object_columns = X_train.select_dtypes(include=['object']).columns.tolist()

preprocessor = ColumnTransformer(
transformers=[
    ('cat', OneHotEncoder(),object_columns)
],remainder='passthrough')

# X_train = preprocessor.fit_transform(X_train)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selection',SelectKBest(k=28)),
    ('classifier', RandomForestClassifier())
])

pipeline.fit(X_train, y_train)


selected_feature_indices = pipeline.named_steps['feature_selection'].get_support()


original_column_names = X_train.columns


selected_features = [col_name for selected, col_name in zip(selected_feature_indices, original_column_names) if selected]

prediction = pipeline.predict(X_test)
test_data['outcome'] = prediction
# print(prediction)
# print("Selected Features:", selected_features)

# output_y = test_data[['id','outcome']]
# output_y.to_csv('outputXX.csv',index=False)


In [ ]:
le = LabelEncoder()
train_data['encoded_outcome'] = le.fit_transform(train_data['outcome'])
y_encoded = train_data['encoded_outcome']

X_train = train_data.drop(['id','outcome','lesion_3','encoded_outcome'],axis=1)
X_test = test_data.drop(['id','lesion_3','outcome'],axis=1)

X_test

dummies_train = X_train.select_dtypes(include='object')
dummies_test = X_test.select_dtypes(include='object')

test_dummies_df = pd.get_dummies(X_test, columns=dummies_test.columns)
train_dummies_df = pd.get_dummies(X_train, columns=dummies_train.columns)

##shapes are misaligned again.  need to compare dataframes

columns_in_train = train_dummies_df.columns
columns_in_test = test_dummies_df.columns

# columns_in_train = [col for col in columns_in_train if col not in columns_in_test]
# print(columns_in_train)##['pain_slight', 'nasogastric_reflux_slight']

# columns_in_test = [col for col in columns_in_test if col not in columns_in_train]
# print(columns_in_test)

def remove_missing_columns(df,column_set):
    for column in column_set:
        if column in df.columns:
            df = df.drop(columns=column)
    return df

missing_columns = ['pain_slight', 'nasogastric_reflux_slight']

test_dummies_df = remove_missing_columns(test_dummies_df,missing_columns)
train_dummies_df = remove_missing_columns(train_dummies_df, missing_columns)




xgb_model = xgb.XGBClassifier(learning_rate=0.5, n_estimator=1000, max_depth=3,
                              objective='binary:logistic')

xgb_model.fit(train_dummies_df,y_encoded)
predictions = xgb_model.predict(test_dummies_df)


feature_importance = xgb_model.feature_importances_

feature_names = train_dummies_df.columns

feature_importance_df = pd.DataFrame({'Feature': feature_names,'Importance': feature_importance})

feature_importance_df = feature_importance_df.sort_values(by='Importance',ascending=False)

plt.figure(figsize=(8,12))
sns.barplot(x='Importance',y='Feature',data=feature_importance_df)
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.title('Feature Importance')
#plt.yticks(rotation=45)
plt.subplots_adjust(left=0.2)
plt.tight_layout
plt.show()



In [ ]:
labels = ['died', 'euthanized', 'lived']

leII = LabelEncoder()
leII.fit(labels)

encoded_label = leII.transform(labels)

labels = leII.inverse_transform(predictions)




test_data['outcome'] = labels


output_y = test_data[['id','outcome']]
output_y.to_csv('outputXXX.csv',index=False)